In [1]:
from tqdm import tqdm

In [2]:
HIGH = 1
LOW = 0

class Module:
    # class variables to track everything
    pulses = {HIGH:0, LOW:0}
    
    def __init__(self, name, targets):
        self.name = name
        self.targets = targets
        
        
    def get_pulse(self, source, level):
        pass
        
    def _send_pulses(self, level):
        Module.pulses[level] += len(self.targets)
        return (self.name, level, self.targets)
    
    def add_source(self, source):
        pass
    
class Button(Module):
    def __init__(self, name='button', targets = ['broadcaster']):
        super().__init__(name=name, targets=targets)
        
    def send_pulse(self):
        return super()._send_pulses(LOW)    
    
class Broadcaster(Module):
    def __init__(self, name='broadcaster', targets = []):
        super().__init__(name=name, targets=targets)
        
    def get_pulse(self, source, level):
        return super()._send_pulses(level)


class FlipFlop(Module):
    def __init__(self, name, targets):
        self.on = False
        super().__init__(name=name, targets=targets)
        
    def get_pulse(self, source, level):
        if level == LOW:
            self.on = not self.on
            if self.on:
                return self._send_pulses(HIGH)
            else:
                return self._send_pulses(LOW)

class Conjunction(Module):
    def __init__(self, name, targets):
        self.memory = {}
        super().__init__(name=name, targets=targets)
        
    def add_source(self, source):
        self.memory[source] = LOW
        
    def get_pulse(self, source, level):
        self.memory[source] = level
        if all(m == HIGH for m in self.memory.values()):
            return self._send_pulses(LOW)
        else:
            return self._send_pulses(HIGH)
    

In [6]:
def reset():
    Module.pulses = {HIGH:0, LOW:0}
    with open('input.txt') as fl:
        lns = fl.readlines()
    config = {'button': Button()}
    for ln in lns:
        source, targets = ln.strip().split(' -> ')
        targets = [t.strip() for t in targets.split(',')]
        if source == 'broadcaster':
            config[source] = Broadcaster(targets = targets)
        elif source.startswith('%'):
            config[source[1:]] = FlipFlop(name=source[1:], targets = targets)
        elif source.startswith('&'):
            config[source[1:]] = Conjunction(name=source[1:], targets = targets)
        else:
            config[source] = Module(name=source, targets=[])
    # now add the sources
    for ln in lns:
        source, targets = ln.strip().split(' -> ')
        targets = [t.strip() for t in targets.split(',')]
        for target in targets:
            if target not in config:
                config[target] = Module(name=source, targets=[])
            if type(config[target]) is Conjunction:
                config[target].add_source(source.strip('%&'))
    return config

In [23]:
def run_next(config, to_run):
    next_round = []
    for source, level, targets in to_run:
        for target in targets:
            next_round.append(config[target].get_pulse(source, level))
        to_run = [v for v in next_round if v is not None]
    return to_run

def run_round(config, stop=None):
    to_run = [config['button'].send_pulse()]
    to_run
    # process as round
    while len(to_run) > 0:
        to_run = run_next(config, to_run)
        if stop is not None:
            for s, l, targets in to_run:
                for t in targets:
                    if s == stop[0] and l == stop[1] and t == stop[2]:
                        return 'STOP'
        #print(to_run)

In [24]:
# part 1
config = reset()
for i in range(1000):
    run_round(config)
Module.pulses[0] * Module.pulses[1]

731517480

In [26]:
# part 2 ... looking at the puzzle, it's clear that these 4 need to be HIGH at the same time
# so find the LCM of the time to get there (and assume they cycle)
score = 1
for source in ['pv','qh','xm','hz']:
    config = reset()
    it = 1
    while run_round(config, stop=(source, HIGH,'kh')) != 'STOP':
        it += 1
    score *= it
score

244178746156661